# Simple neural network for MBL-ETH Transition

In [1]:
from __future__ import print_function
import sklearn
# suppress tensorflow compilation warnings
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
seed=0
np.random.seed(seed) # fix random seed
# tf.set_random_seed(seed)
import matplotlib.pyplot as plt
%config InlineBackend.figure_format ='retina' #plot high-res img 

## Use entanglement spectrum to feed the network
$j=5.0$ data represent typical the thermal phase, $j=0.1$ for the MBL phase 

In [2]:
# Load the example data set with L=12
# p1 represents the localized MBL phase while p2 is ETH phase
x = np.load('phase_1.npz')
p1 = x['ent']
x = np.load('phase_2.npz')
p2 = x['ent']
print(p1.shape,p2.shape)

(6560, 128) (6560, 128)


In [ ]:
# This is only useful if you run the en_spectrum.py script to generate your own data set
L = 12
runs = 4 # number of samples generated
loaded = np.load('1en_spectrum_L=12_j=0.1.npz')

dim= len(loaded['ent']) # the dim is determined by the cut position
x = np.zeros((runs,dim,2**(int(L//2+1))), dtype='float64')
for i in range(runs):
    filename = ""+str(i+1)+"en_spectrum_L=12_j=0.1.npz"
    loaded = np.load(filename)
    x[i] = loaded['ent']
p1 = np.concatenate((x[:])) 

runs = 4
x = np.zeros((runs,dim,2**(int(L//2+1))), dtype='float64')
for i in range(runs):
    filename = ""+str(i+1)+"en_spectrum_L=12_j=5.0.npz"
    loaded = np.load(filename)
    x[i] = loaded['ent']
p2 = np.concatenate((x[:])) 

# cast the original data to single precision to speed up
p1 = p1.astype('float32')
p2 = p2.astype('float32')
print(p1.shape, p2.shape)

## Use wavefunction (absolute value squared) as raw data to feed the network

In [ ]:
# Load the wavefunction data
x = np.load('phase_1.npz')
p1 = x['wav']
x = np.load('phase_2.npz')
p2 = x['wav']
print(p1.shape,p2.shape)

In [ ]:
# This is only useful if you run the en_spectrum.py script to generate your own data set
L = 12
runs = 4
dim = int(2**(L-1))
x = np.zeros((runs, 200, dim), dtype='float64')
for i in range(runs):
    filename = ""+str(i+1)+"en_spectrum_L=12_j=0.1.npz"
    loaded = np.load(filename)
    wav = loaded['wave']
    abswav = np.multiply(wav,np.conj(wav))
    x[i] = np.multiply(wav,np.conj(wav))
p1 = np.concatenate((x[:]))

runs = 4
x = np.zeros((runs, 200, dim), dtype='float64')
for i in range(runs):
    filename = ""+str(i+1)+"en_spectrum_L=12_j=5.0.npz"
    loaded = np.load(filename)
    wav = loaded['wave']
    abswav = np.multiply(wav,np.conj(wav))
    x[i] = np.multiply(wav,np.conj(wav))
p2 = np.concatenate((x[:]))

p1 = p1.astype('float32')
p2 = p2.astype('float32')
print(p1.shape, p2.shape)

### Label two phases and partition trainning and test set

In [13]:
from sklearn.model_selection import train_test_split

#combine entanglement spectrum of different system sizes with 2 labels
num_classes = 2
l1 = np.ones(p1.shape[0],dtype=np.int8)
l2 = np.zeros(p2.shape[0],dtype=np.int8)
ph1 = np.column_stack((l1,p1))
ph2 = np.column_stack((l2,p2))
# gathering all the data
dat = np.concatenate((ph1,ph2))
X_train, X_test, Y_train, Y_test = train_test_split(dat[:,1:], dat[:,:1], test_size=0.2, random_state=1)

# convert class vectors to binary class matrices
Y_train = tf.keras.utils.to_categorical(Y_train, num_classes)
Y_test = tf.keras.utils.to_categorical(Y_test, num_classes)
print(X_train.shape, Y_train.shape, Y_test.shape)

(10496, 128) (10496, 2) (2624, 2)


## Create the neural network

In [18]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization

def create_model():
    # instantiate model
    model = Sequential()
    # add a normalization layer to improve robustness
    model.add(BatchNormalization(axis=-1, momentum=0.95, epsilon=0.001))
    # add a dense all-to-all relu layer
    model.add(Dense(400,input_shape=(X_test.shape[1],), activation='relu'))
    # add a dense all-to-all relu layer
    model.add(Dense(50, activation='relu'))
    # apply dropout with rate 0.4
    model.add(Dropout(0.4))
    # soft-max layer
    model.add(Dense(num_classes, activation='softmax'))
    
    # compile the model with loss function and optimizer
    model.compile(loss=tf.keras.losses.binary_crossentropy,
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

### Train the model

In [19]:
# training parameters
batch_size = 150
epochs = 10

from tensorflow.keras.callbacks import EarlyStopping
#early termination to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# create the deep neural net
model_DNN=create_model()

# train DNN and store training info in history
history=model_DNN.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test),callbacks=[early_stopping])

Train on 10496 samples, validate on 2624 samples
Epoch 1/10
10496/10496 [==============================] - 2s 203us/sample - loss: 0.4057 - accuracy: 0.8096 - val_loss: 2.5493 - val_accuracy: 0.5252
Epoch 2/10
10496/10496 [==============================] - 1s 58us/sample - loss: 0.1958 - accuracy: 0.9224 - val_loss: 0.1499 - val_accuracy: 0.9379
Epoch 3/10
10496/10496 [==============================] - 1s 55us/sample - loss: 0.1444 - accuracy: 0.9424 - val_loss: 0.1340 - val_accuracy: 0.9466
Epoch 4/10
10496/10496 [==============================] - 1s 54us/sample - loss: 0.1150 - accuracy: 0.9516 - val_loss: 0.1458 - val_accuracy: 0.9398
Epoch 5/10
10496/10496 [==============================] - 1s 55us/sample - loss: 0.0990 - accuracy: 0.9603 - val_loss: 0.1499 - val_accuracy: 0.9398
Epoch 6/10
10496/10496 [==============================] - 1s 55us/sample - loss: 0.0807 - accuracy: 0.9705 - val_loss: 0.1397 - val_accuracy: 0.9459


#### Optional: Save trained model which can be trained later

In [ ]:
#save the model and continue training
model_DNN.save('my_model.h5')
from tensorflow.keras.models import load_model
model_DNN = load_model('my_model.h5')


# training parameters
batch_size = 100
epochs = 10

# create the deep neural net
# model_DNN=compile_model()

# train DNN and store training info in history
history = model_DNN.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test),callbacks=[early_stopping])

### Step 5: Evaluate the Model Performance on the *Unseen* Test Data

Next, we evaluate the model and read of the loss on the test data, and its accuracy using the `evaluate()` method.

In [20]:
history.history['acc']

KeyError: 'acc'

In [ ]:
# evaluate model
score = model_DNN.evaluate(X_test, Y_test, verbose=1)

# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# look into training history

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.ylabel('model accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('model loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

### Hyperparameters tuning
Use scikit-learn to grid search the batch size and epochs

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [20, 50, 100]
epochs = [5, 10, 15]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))